## Getting to follow a red spot

In [ ]:
import cv2
import numpy as np

# Start video capture (0 for webcam or drone feed URL if available)
# The 0 needs to be changed with the url of the drones camera
# This will open the camera that I will be using 
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to HSV for better color detection
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Define red color range (tune values as needed)
    lower_red = np.array([0, 120, 70])
    upper_red = np.array([10, 255, 255])

    # Create a mask for red color
    mask = cv2.inRange(hsv_frame, lower_red, upper_red)

    # Find contours of the red object
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    # For each contour 
    for contour in contours:
        # Calculate area of the contour and filter noise 
        area = cv2.contourArea(contour)
        # Minimum size to filter noise
        if area > 100:  
            # Draw a circle around the centroid of the contour
            x, y, w, h = cv2.boundingRect(contour)
            # Draw rectangle around the red object
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            # Print the position of the red object
            print(f"Red spot position: x={x}, y={y}")

            # Determine position relative to the center of the frame
            frame_center_x, frame_center_y = frame.shape[1] // 2, frame.shape[0] // 2
            # //2 Rounds down this can be changed according the the implementation 
            offset_x = x + w//2 - frame_center_x
            offset_y = y + h//2 - frame_center_y

            # Use offset to decide drone movement (e.g., up, down, left, right)
            print(f"Offset: x={offset_x}, y={offset_y}")

    # Display the video feed
    cv2.imshow("Red Spot Tracking", frame)

    # Break loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [1]:
import requests

API_URL = "https://api.dji.com/v1/control"
API_KEY = "your_api_key_here"

def send_movement_command(yaw, pitch, roll, throttle):
    payload = {
        "yaw": yaw,
        "pitch": pitch,
        "roll": roll,
        "throttle": throttle
    }
    headers = {"Authorization": f"Bearer {API_KEY}"}
    response = requests.post(API_URL, json=payload, headers=headers)
    if response.status_code == 200:
        print("Command sent successfully.")
    else:
        print(f"Error: {response.status_code} - {response.text}")

# Example offsets translated into movement
send_movement_command(0, offset_y / MAX_FRAME_HEIGHT, offset_x / MAX_FRAME_WIDTH, 0)


NameError: name 'offset_y' is not defined

## Getting the drone to track a person (either as a person or maybe accessory like the hat with a cross thing he was talking about) and hover behind them wherever they go (slow movement)

### Note that the contour changes the shape of the thing that we are searching from here I changed the shape of the thing to be searched from.

In [3]:
import cv2
import numpy as np

# Start video capture (change 0 to camera URL if using drone)
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to HSV for better color filtering
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Define red color range (tweak these values if needed)
    lower_red1 = np.array([0, 120, 70])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([170, 120, 70]) 
    upper_red2 = np.array([180, 255, 255])

    # Create masks for red color
    mask1 = cv2.inRange(hsv_frame, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv_frame, lower_red2, upper_red2)
    mask = mask1 + mask2  # Combine both masks

    # Apply morphological operations to clean up noise
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

    # Find contours in the red mask
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        area = cv2.contourArea(contour)
        if area > 500:  # Ignore small objects
            # Get bounding box
            x, y, w, h = cv2.boundingRect(contour)
            
            # Aspect ratio check (Crosses tend to have width ≈ height)
            aspect_ratio = float(w) / h
            # Aspect ratio for a cross
            # Changed this to represent a cross 
            if 0.5 < aspect_ratio < 1.5:  
                # Draw rectangle around detected cross
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                
                # Find center of the cross
                center_x, center_y = x + w // 2, y + h // 2
                
                # Print detected cross position
                print(f"Red Cross Found at: x={center_x}, y={center_y}")

                # Find offset from center of the frame
                # //2 Rounds down this can be changed according the the implementation 
                frame_center_x, frame_center_y = frame.shape[1] // 2, frame.shape[0] // 2
                offset_x = center_x - frame_center_x
                offset_y = center_y - frame_center_y

                # Use offset to decide drone movement (e.g., up, down, left, right)
                print(f"Offset: x={offset_x}, y={offset_y}")

    # Display the processed frame
    # Display the video feed
    # cv2.imshow("Red Cross Detection", frame)

    # Break loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Red Cross Found at: x=1197, y=684
Offset: x=557, y=324
Red Cross Found at: x=758, y=654
Offset: x=118, y=294
Red Cross Found at: x=1206, y=625
Offset: x=566, y=265
Red Cross Found at: x=966, y=499
Offset: x=326, y=139
Red Cross Found at: x=1010, y=450
Offset: x=370, y=90
Red Cross Found at: x=1198, y=694
Offset: x=558, y=334
Red Cross Found at: x=81, y=337
Offset: x=-559, y=-23
Red Cross Found at: x=286, y=298
Offset: x=-354, y=-62
Red Cross Found at: x=739, y=643
Offset: x=99, y=283
Red Cross Found at: x=957, y=665
Offset: x=317, y=305
Red Cross Found at: x=1093, y=607
Offset: x=453, y=247
Red Cross Found at: x=1047, y=284
Offset: x=407, y=-76
Red Cross Found at: x=1107, y=245
Offset: x=467, y=-115
Red Cross Found at: x=960, y=250
Offset: x=320, y=-110
Red Cross Found at: x=1237, y=226
Offset: x=597, y=-134
Red Cross Found at: x=144, y=319
Offset: x=-496, y=-41
Red Cross Found at: x=295, y=202
Offset: x=-345, y=-158
Red Cross Found at: x=320, y=180
Offset: x=-320, y=-180
Red Cross Fou

KeyboardInterrupt: 

Litreature 
- video of drone Face tracking part 
https://www.youtube.com/watch?v=LmEcyQnfpDA

- cv2 github repo to fetch the datasets


In [ ]:
# FACE TRACKING - this works very well

import cv2
import numpy as np

# Detecting a face
def findFace(img):
    faseCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = faseCascade.detectMultiScale(imgGray, 1.2, 8)

    myFaceList = []
    myFaceListArea = []

    for(x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), 2)
        cx = x + w//2
        cy = y + h//2
        area = w*h
        cv2.circle(img, (cx, cy), 5, (0, 255, 0), cv2.FILLED)
        myFaceList.append((cx, cy, area))
        myFaceListArea.append(area)
    
    if len(myFaceList)!= 0:
        i = myFaceListArea.index(max(myFaceListArea))
        return img, [myFaceList[i], myFaceListArea[i]]
    
    # Default return to prevent errors
    return img, [[0, 0], 0]
    
# def trackFace(me, info, w, pidf, pError):
#     return


cap = cv2.VideoCapture(0)

while True:
    _, img = cap.read()
    img, info = findFace(img)
    print(f'Area: {info[1]}, Center: {info[0]}')
    cv2.imshow("Face Tracking", img)
    cv2.waitKey(1)

In [ ]:
# This is detecting clocks as well as people

import cv2
import numpy as np

# Detecting a person
def findPerson(img):
    personCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_upperbody.xml")
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    persons = personCascade.detectMultiScale(imgGray, 1.1, 4)  # Adjust scaleFactor & minNeighbors if needed

    myPersonList = []
    myPersonListArea = []

    for (x, y, w, h) in persons:
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), 2)
        cx = x + w // 2
        cy = y + h // 2
        area = w * h
        cv2.circle(img, (cx, cy), 5, (0, 255, 0), cv2.FILLED)
        myPersonList.append((cx, cy))
        myPersonListArea.append(area)

    if len(myPersonList) != 0:
        i = myPersonListArea.index(max(myPersonListArea))  # Select largest detected person
        return img, [myPersonList[i], myPersonListArea[i]]

    return img, [[0, 0], 0]  # Default return to prevent errors

cap = cv2.VideoCapture(0)

while True:
    _, img = cap.read()
    img, info = findPerson(img)
    print(f'Area: {info[1]}, Center: {info[0]}')

    cv2.imshow("Person Detection", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break  # Press 'q' to exit

cap.release()
cv2.destroyAllWindows()


In [2]:
import requests

DJI_ACCESS_TOKEN = "your_dji_cloud_api_access_token"
MEDIA_URL = "https://api.dji.com/v1/media"

headers = {"Authorization": f"Bearer {DJI_ACCESS_TOKEN}"}
response = requests.get(MEDIA_URL, headers=headers)

if response.status_code == 200:
    print(response.json())  # List of media files on the drone
else:
    print("Error accessing DJI Cloud API:", response.text)


ConnectionError: HTTPSConnectionPool(host='api.dji.com', port=443): Max retries exceeded with url: /v1/media (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x108814d50>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

Challenges and Considerations -- things to check 
- Latency: Ensure minimal delay between detecting the red spot and sending commands.
- Stability: Fine-tune thresholds for red detection to avoid false positives.
- Safety: Test in a controlled environment to ensure predictable movements.
- Camera Feed Access: If using the drone’s camera feed, ensure you can stream it to your processing device.
